## Accessing Reference Designator Annotations via the API

*Compiled by Lori Garzio, Rutgers University, July 13, 2018*

*Modified from Section 3d in OOI_Example_Data_Quality_Report.ipynb written by Sage Lichtenwalner*

This provides a quick example of accessing annotations for a specific instrument (reference designator). The API will return all associated annotations for the instrument specified, including annotations entered at the subsite, node, and data stream level.

In [ ]:
import datetime
import requests
import pandas as pd

In [ ]:
USERNAME = ''
TOKEN = ''

# Specify a reference designator
refdes = 'GI01SUMO-SBD12-08-FDCHPA000'

# Specify a time range of interest
# begin = int(datetime.date(2017,1,1).strftime('%s'))*1000
# end = int(datetime.date(2018,1,1).strftime('%s'))*1000

# Alternatively, if you want all annotations, use these dates
begin = int(datetime.date(2013,1,1).strftime('%s'))*1000
end = int(datetime.datetime.now().strftime("%s")) * 1000 # current date

In [ ]:
ANNO_API = 'https://ooinet.oceanobservatories.org/api/m2m/12580/anno/find'
params = {
  'beginDT': begin,
  'endDT': end,
  'refdes': refdes,
}

# Send the request
r = requests.get(ANNO_API, params=params, auth=(USERNAME, TOKEN))
data = r.json()

In [ ]:
# convert timestamps
def convert_time(ms):
    if ms != None:
        return datetime.datetime.utcfromtimestamp(ms/1000)
    else:
        return None

In [ ]:
# Convert info the json response to a dataframe for easier visualization
df = pd.DataFrame() # Setup empty array
for d in data:
    df = df.append({
        'annotation_id': d['id'],
        'subsite': d['subsite'],
        'node': d['node'],
        'sensor': d['sensor'],
        'stream': d['stream'],
        'method': d['method'],
        'start': convert_time(d['beginDT']),
        'stop': convert_time(d['endDT']),
        'qcFlag': d['qcFlag'],
        'annotation': d['annotation']
    }, ignore_index=True)
pd.set_option('display.max_colwidth', -1) # Show the full annotation text
df

In [ ]:
# Write annotations to a csv file in working directory
df.to_csv('{}_annotations.csv'.format(refdes), index=False)